In [ ]:
# 보고서 생성 : 이미지 추가
# 출처 : https://www.slideshare.net/plusjune/ss-110416117
# 슬라이드 내 37page

In [4]:
###############
# 1. import
#####

import pandas as pd
from datetime import datetime, timedelta
import re
import time 

# local only
import os
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)

In [5]:
###############
# 2.자료 추출
#####

# DB연결
conn = pymysql.connect(host = '**.***.*.*', user ='__USER_ID__', password = "__PASSWORD__", database='__DATABAES__')

# cursor 설정
cursor = conn.cursor(pymysql.cursors.DictCursor)


#자료 불러오기 : 최근 2주간 CS 미전송 껀
sql= '''select count(distinct CONTENTS) as CNT
        FROM MAEIL_REVIEW A
        where GRADE in (1,2)
        and WRITE_DT between (CURDATE()-1) and (CURDATE())
        '''
cursor.execute(sql)
result0 = cursor.fetchall()
result0 = pd.DataFrame(result0)

In [6]:
#자료 불러오기 : 최근 2주간 CS 미전송 껀
sql= '''select (SELECT CODE_NM FROM JT_CODE B WHERE A.COMPANY_CODE = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') as 수집체널 
        , GROUP_CONCAT((SELECT CODE_NM FROM JT_CODE B WHERE A.PRODUCT_CODE = B.CODE AND B.CODE_GRP_ID = 'PRODUCT') SEPARATOR ', ') as 구매제품
        , A.WRITER as "작성자ID" , A.CONTENTS as 내용 , A.WRITE_DT as 작성일자 , A.GRADE as "평점(5점 만점)"
        FROM MAEIL_REVIEW A
        where GRADE in (1,2)
        and A.COMPANY_CODE in (13261, 13358)
        and WRITE_DT between (CURDATE()-1) and (CURDATE())
        and PRODUCT_CODE not like 'G_%'
        group by A.COMPANY_CODE, A.WRITER, A.CONTENTS, A.WRITE_DT, A.GRADE
        order by A.COMPANY_CODE, A.GRADE desc
        '''
cursor.execute(sql)
result5 = cursor.fetchall()
# result = cursor.fetchmany(3)
result5 = pd.DataFrame(result5)

date2 = datetime.today().strftime('%Y%m%d')
excel_name = date2 +'_review_data.xlsx'
result5.to_excel(excel_name, sheet_name= 'Sheet1' )

naver_count = len(result5[result5['수집체널'] =='25 카카오 톡스토어(20.12.2~미사용)'])
kakao_count = len(result5[result5['수집체널'] =='3 네이버'])

In [7]:
#자료 불러오기 : 최근 2주간 CS 미전송 껀
sql= '''select (SELECT CODE_NM FROM JT_CODE B WHERE A.COMPANY_CODE = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') as 수집체널 
        , GROUP_CONCAT(distinct (SELECT CODE_NM FROM JT_CODE B WHERE A.PRODUCT_CODE = B.CODE AND B.CODE_GRP_ID = 'PRODUCT') SEPARATOR ', ') as 구매제품
        , A.WRITER as "작성자ID" , A.CONTENTS as 내용 , A.WRITE_DT as 작성일자 , A.GRADE as "평점(5점 만점)"
        FROM MAEIL_REVIEW A
        where GRADE in (1,2)
        and A.COMPANY_CODE in (13261, 13358)
        and WRITE_DT between (CURDATE()-1) and (CURDATE())
        and PRODUCT_CODE not like 'G_%'
        group by A.COMPANY_CODE, A.WRITER, A.CONTENTS, A.WRITE_DT, A.GRADE
        order by A.COMPANY_CODE, A.GRADE desc
        '''
cursor.execute(sql)
result5 = cursor.fetchall()
# result = cursor.fetchmany(3)
result5 = pd.DataFrame(result5)

date2 = datetime.today().strftime('%Y%m%d')
excel_name = date2 +'_review_data.xlsx'
result5.to_excel(excel_name, sheet_name= 'Sheet1' )

naver_count = len(result5[result5['수집체널'] =='25 카카오 톡스토어(20.12.2~미사용)'])
kakao_count = len(result5[result5['수집체널'] =='3 네이버'])

In [3]:
###############
# 3.이메일 발송(module)
#####
import pandas as pd
import os
import smtplib
# from email.headerregistry import Address
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email import encoders
from email.header import Header
from datetime import datetime, timedelta

from jinja2 import Template

gmail_user= '******@maeil.com'
gmail_pwd = '************'

def send_email(to, cc, subject, html, attach=None):
    msg=MIMEMultipart('related')
    msg['From']=gmail_user
    msg['To'] = ", ".join(to)
    msg['Cc'] = ", ".join(cc)
    msg['Subject'] = Header(s=subject, charset="utf-8")
    msg.attach(MIMEText(html, 'html', _charset="utf-8"))

    #첨부파일
    if attach:
        part=MIMEBase('application','octet-stream')
        part.set_payload(open(attach, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition','attachment; filename="%s"' % os.path.basename(attach))
        msg.attach(part)
     
    s = smtplib.SMTP("mail.maeil.com", 25)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login(gmail_user, gmail_pwd)
    s.sendmail(gmail_user, to, msg.as_string())
    s.close()

In [8]:


# 이메일 양식(내용) frame 구성하기
message_tmpl="""
<html>
    <!DOCTYPE html>
    <html>
    <head lang ="ko">
      <meta charset ="UTF-8">
      <title>{{title1}}</title>
    </head>
    <BODY style="FONT-SIZE: 10pt; font-family: 맑은고딕, Malgun Gothic, dotum, gulim, sans-serif">
    안녕하세요,<br>
    <br>
    {{send_date}}시 기준, 직영채널의 부정리뷰 현황 내용 공유드립니다.<br>
    (해당 메일의 내용 문의는 발송계정이 아닌, 박경하 대리에게 말씀 부탁드립니다.)<br>
    <br>
    <b>1. 전체 직영 채널 부정 리뷰 : {{count1}} 개</b><br>
    <br>
    <b>2. 관리 가능 채널 부정 리뷰 : {{count2}} 개</b><br>
    &nbsp;&nbsp;- 네이버  : {{ count3 }} 개<br>
    &nbsp;&nbsp;- 카카오 톡스토어  : {{ count4 }} 개<br>
    <br>
    <b>3. 부정리뷰 List</b><br>
    {{ df_html }}<br>
    <br>
    <SPAN style="COLOR: #808080">{{title3_text2}}</SPAN><br>
    <br>
    감사합니다.<br>
    <br>
    </BODY>
</html>"""


render_data = {
    'send_date' : (datetime.today()).strftime('%Y-%m-%d, %H'),
    'df_html' : result5.to_html(),
    'count1' : result0['CNT'][0],
    'count2' : int(naver_count) + int(kakao_count),
    'count3' : naver_count,
    'count4' : kakao_count,
    'title3_text2' : "* 본 메일은 발신전용입니다."
}

t= Template(message_tmpl)
html_report = t.render(render_data)


In [9]:


# 발송 변수 설정

# 받는사람 : test 발송
to_users = ['khpark@maeil.com']

# 참조 : test 발송
cc_users = ['khpark@maeil.com']

# 받는사람 : mulit 발송
# to_users = ['khpark@maeil.com', 'cappygirl@maeil.com', 'soykim10@maeil.com', 'jinto170@maeil.com', 'pigangel@maeil.com', 'k8802@maeil.com', 'billy@maeil.com' ]

# 메일제목
now = datetime.today()    # 현재
date = now.strftime('%Y%m%d')

subject = "[자동발송] 고객 부정리뷰 관리 List(TEST_v3)"

# 본문
html = html_report

# 첨부파일 추가
attach = './' + excel_name



In [11]:
#메일 발송
send_email(to_users, '', subject, html, attach)